In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras
from keras.layers import Dense, Dropout, Input
from keras.models import Model,Sequential
from keras.datasets import mnist
from tqdm import tqdm
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import adam

In [ ]:
def load_data():
    (x_train,y_train),(x_test,y_test) = mnist.load_data()
    x_train = (x_train.astype(np.float32)-127.5)/127.5
    x_train = x_train.reshape(60000,784)
    return(x_train,y_train,x_test,y_test)

(X_train,Y_train,X_test,Y_test) = load_data()
print(X_train.shape)
    

10829824/11490434 [===========================>..] - ETA: 0s